In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv("fruits.csv")

In [3]:
color = pd.get_dummies(df.color)
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
df.target = le1.fit_transform(df.target)
df.color = le1.fit_transform(df.color)

In [4]:
from sklearn.preprocessing import KBinsDiscretizer
kb = KBinsDiscretizer(n_bins=5000, encode='onehot-dense')
W = kb.fit_transform(df[["weight"]])


In [5]:
df = pd.concat([df, color, pd.DataFrame(W)], axis = 1)

In [6]:
df

,target,weight,color,green,orange,red,0,1,2,3,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0,91.234258,2,False,False,True,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,125.134927,2,False,False,True,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,101.313929,1,False,True,False,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,102.218174,0,True,False,False,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,89.982126,2,False,False,True,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,81.061794,2,False,False,True,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,1,90.426004,1,False,True,False,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,1,92.361074,1,False,True,False,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0,126.969072,0,True,False,False,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
names = [x for x in range(5000)]
X = df[["color"]+names]
y = df.target

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=100.0, max_iter=1000)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
from sklearn.model_selection import KFold
pred_test = np.zeros((X_test.shape[0], 5))
kf = KFold(n_splits=5)
scores = []
for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    X_tr = X_train.iloc[train_index, :]
    y_tr = y_train.iloc[train_index]
    X_val = X_train.iloc[test_index, :]
    y_val = y_train.iloc[test_index]
    clf.fit(X_tr, y_tr)
    pred_tr = clf.predict_proba(X_tr)[:,1]
    pred_val = clf.predict_proba(X_val)[:,1]
    pred_test[:,i] = clf.predict_proba(X_test)[:,1]
    score_tr = roc_auc_score(y_tr, pred_tr)
    score_val = roc_auc_score(y_val, pred_val)
    scores.append(score_val)
    print(score_tr, score_val)


0.9942565198814314 0.8292405853165962
0.995696208945747 0.8133395655270655
0.9941699169916991 0.8253798123060434
0.9949483977790848 0.8072901846687914
0.9955952421702703 0.8079414081741892


In [18]:
clf.fit(X_train, y_train)
pred_train = clf.predict_proba(X_train)[:,1]
pred_test = clf.predict_proba(X_test)[:,1]
score_train = roc_auc_score(y_train, pred_train)
score_test = roc_auc_score(y_test, pred_test)
print(score_train, score_test)

0.9931644234921371 0.8442791350812536


In [11]:
from scipy.stats import bootstrap
bootstrap([scores], np.mean, confidence_level=0.9).confidence_interval.low

0.8109198131837558

In [12]:
# 10000 0.676
# 5000 0.773
# 1000 0.908
# 500 0.905

#C=1 0.768770986002693
#C=10 0.8000324911692527
#C=100 0.8109198131837558

In [14]:
pred_test.mean(axis=1)

array([0.32508251, 0.13722167, 0.97428017, ..., 0.32508251, 0.63155033,
       0.81170973])

In [15]:
score_test = roc_auc_score(y_test, pred_test.mean(axis=1))

In [16]:
score_test

0.8457176553944672

In [17]:
for i in range(5):
    score_test = roc_auc_score(y_test, pred_test[:,i])
    print(score_test)

0.8196644541662637
0.819297697150314
0.8189924400007943
0.8247644587146912
0.8176743569260062
